### DOY Copernicus

Script for change the value for the pixel to DOY. Applicable only for VPP image

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd

import shapely.wkt
import shapely.geometry
from shapely import wkt
import rasterio as rio
import collections
collections.Callable = collections.abc.Callable

import warnings
warnings.filterwarnings('ignore')

In [3]:
# year=2020
tipo='MAX'

y1=17
# y2=20

fn='D:/Documents/Copernicus/VPP_%(y)s_%(tipo)s_wwSINDOY.tif'%{'y':y1,'tipo':tipo}
with rio.open(fn, 'r') as file:
    data = file.read()
    data_metadata = file.meta
    file.close

print(data_metadata)
print(data.shape)

{'driver': 'GTiff', 'dtype': 'int16', 'nodata': 0.0, 'width': 31407, 'height': 34547, 'count': 1, 'crs': CRS.from_epsg(4326), 'transform': Affine(0.00015561216773330783, 0.0, 10.262301292,
       0.0, -8.51689131617797e-05, 54.221367684)}
(1, 34547, 31407)


In [4]:
imagen=data[0,:,:]
# imagen.shape

In [5]:
smallest = imagen.min()
print(smallest)
largest = imagen.max()
print(largest)

0
17365


In [6]:
data_mask = np.ma.masked_values(imagen, 0)

In [8]:
count1 = np.count_nonzero(~data_mask.mask)
print('Print count of True elements in array1: ', count1)

Print count of True elements in array1:  17283811


In [9]:
smallest = data_mask.min()
print(smallest)
largest = data_mask.max()
print(largest)

17001
17365


### cambio de valores de pixeles

In [11]:
data_prueba = imagen.reshape(-1)
# data_prueba_mask = data_mask.mask.reshape(-1)

print(data_prueba.shape[0])

1085017629


In [15]:
%%time

valores_y1=[]
# valores_y2=[]

# bloque=int(data_prueba.shape[0]/c)

# for k in range(c-1):bloque*k,bloque*(k+1)

for i in range(data_prueba.shape[0]):    
#     if data_prueba[i] != 0:
    valor=int(data_prueba[i]/1000)
    if valor==y1:
        valores_y1.append(data_prueba[i]-(valor*1000))
#             valores_y2.append(0)
#         if valor==y2:
#             valores_y2.append(data_prueba[i]-(valor*1000))
#             valores_y1.append(0)
    else:
        valores_y1.append(0)
#         valores_y2.append(0)

CPU times: total: 52min 53s
Wall time: 53min 1s


In [16]:
datafinal_y1 = np.reshape(valores_y1, (imagen.shape[0], imagen.shape[1]))
# datafinal_y2 = np.reshape(valores_y2, (imagen.shape[0], imagen.shape[1]))

In [17]:
smallest = np.nanmin(datafinal_y1)
print(smallest)
largest = np.nanmax(datafinal_y1)
print(largest)

0
365


In [22]:
valores_y1=0
valores_y2=0

In [23]:
from rasterio.crs import CRS

# for y in [y1,y2]:
output=r'D:\Documents\Copernicus\clip_poligono_DOY_wheat\VPP_%(y)s_%(tipo)s_ww.tif'%{'y':y1,'tipo':tipo}
#     if y==y1:
datafinal=datafinal_y1
#     if y==y2:
#         datafinal=datafinal_y2

with rio.open(output,
              mode="w",driver="GTiff",
              height=datafinal.shape[0],
              width=datafinal.shape[1],
              count=1,
              dtype=datafinal.dtype,
              crs=CRS.from_epsg(4326),
              transform=data_metadata['transform'],
) as new_dataset:
        new_dataset.write(datafinal, 1)